In [5]:
work_dir = "/home/kpatelis/projects/bggrecommender/"

In [7]:
!pip install bs4

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 7.0 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=d70e5c8401d18e3106741ca741535530c974f875e076467edd73a83e95ca8941
  Stored in directory: /home/kpatelis/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [8]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import polars as pl
import numpy as np
#import pickle
from time import sleep

In [9]:
username='kage21'
password='GP;b:zo-+63g'

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Path to the ChromeDriver executable
driver_path = '/home/kpatelis/projects/bggrecommender/chromedriver/chromedriver'

# Configure Chrome options
options = Options()
#options.binary_location = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"
options.add_argument('--headless')  # Run Chrome in headless mode

# Set up the Chrome driver
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

In [12]:
login_url = 'https://www.boardgamegeek.com/login'
driver.get(login_url)

# Find and interact with the login form elements
username_input = driver.find_element(By.NAME, 'username')
password_input = driver.find_element(By.NAME, 'password')
login_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')

# Enter the username and password
username_input.send_keys(username)
password_input.send_keys(password)

# Click the login button
login_button.click()

In [60]:
# Scanned on 20230607

df_all = pd.DataFrame(columns=["id", "name", "rank"])
npage = 1
min_rank = 1e5
max_rank = 1e6

while min_rank != max_rank:
    #r = request(f"https://boardgamegeek.com/browse/boardgame/page/{npage}?sort=numvoters&sortdir=desc", driver=driver)
    driver.get(f"https://boardgamegeek.com/browse/boardgame/page/{npage}?sort=rank&sortdir=asc")
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    table = soup.find_all("tr", attrs={"id": "row_"})
    
    df = pd.DataFrame(columns=["id", "name", "rank"], index = range(len(table)))
    
    for idx, row in enumerate(table):
        links = row.find_all("a")
        if "name" in links[0].attrs.keys():
        #    del links[0]
            rank = int(links[0]["name"])
            gamelink = links[2] # Get relative url of each game
            gameid = int(gamelink["href"].split("/")[2]) # Get game id
            gamename = gamelink.contents[0] # Get the actual name of the game
        #imlink = links[0] # Get url for thumbnail
        #thumbnail = imlink.contents[0]["src"]
        
        #ratings_str = row.find_all("td", attrs = {"class":"collection_bggrating"})[2].contents[0]
        #nratings = int("".join(ratings_str.split()))
        
        df.iloc[idx, :] = [gameid, gamename, rank]
    min_rank = df["rank"].min()
    max_rank = df["rank"].max()

    if min_rank != max_rank:
        print(f"Page {npage} scraped, games betweens ranks {min_rank} and {max_rank}")
        df_all = pd.concat([df_all, df], axis=0)
        npage += 1
    
    sleep(2)
    
driver.quit()

df_all = df_all.drop_duplicates()
df = df_all.copy()
df.reset_index(inplace=True, drop=True)
df.to_csv(f"{work_dir}/data/bgg_gamelist_tmp.csv", index=False, encoding="utf-8")

Page 244 scraped, games betweens ranks 24301 and 24400
Page 245 scraped, games betweens ranks 24401 and 24487


In [2]:
def request(msg, slp=2):
    '''A wrapper for robust https requests'''
    status_code = 500 
    while status_code != 200:
        sleep(slp) # Avoid pinging the server too often to not get blacklisted
        try:
            r = requests.get(msg)
            status_code = r.status_code
            if status_code != 200:
                print(f"Server Error! Response code {status_code}. Retrying...")
        except:
            print("An exceptions has occurred, probably momentary loss of connection. Waiting for a second...")
            sleep(1)
    return r

In [6]:
# Scanned on 20230607

start_time = time.time()

df_games = pd.DataFrame(columns=["id", "name", "image", "thumbnail", "description", "min_players", 
                                 "max_players", "playing_time", "is_expansion", "year_published", 
                                 "bgg_rating", "avg_rating", "rank", "mechanics", "designers", "artists", 
                                 "publishers", "expansions"])

base_url = 'https://bgg-json.azurewebsites.net/'
game_ids = pd.read_csv(f"{work_dir}/data/bgg_gamelist_all.csv")["id"].sort_values()

for game_id in game_ids:
    r = request(f'{base_url}/thing/{game_id}')
    data = r.json()
    
    df_games.loc[len(df_games)] = {
        "id": data["gameId"],
        "name": data["name"],
        "image": data["image"],
        "thumbnail": data["thumbnail"],
        "description": data["description"],
        "min_players": data["minPlayers"], 
        "max_players": data["maxPlayers"], 
        "playing_time": data["playingTime"], 
        "is_expansion": data["isExpansion"],
        "year_published": data["yearPublished"], 
        "bgg_rating": data["bggRating"], 
        "avg_rating": data["averageRating"], 
        "rank": data["rank"], 
        "mechanics": ', '.join(data["mechanics"]),
        "designers": ', '.join(data["designers"]), 
        "artists": ', '.join(data["artists"]), 
        "publishers": ', '.join(data["publishers"]), 
        "expansions": ', '.join([str(expansion["gameId"]) + "__" + expansion["name"] for expansion in data["expansions"]]) if "expansions" in data else "None"
    }
    
    name = data["name"]
    print(f"Data for board game {name} with game ID: {game_id} acquired.")
    
    sleep(2)

end_time = time.time()
diff_time = end_time - start_time

print(f"Scraping took {diff_time} seconds to complete.")

df_games.reset_index(inplace=True, drop=True)
df_games.to_csv(f"{work_dir}/data/bgg_gamelist_all_details_tmp.csv", index=False, encoding="utf-8")

Data for board game Die Macher with game ID: 1 acquired.
Data for board game Dragonmaster with game ID: 2 acquired.
Data for board game Samurai with game ID: 3 acquired.
Data for board game Tal der Könige with game ID: 4 acquired.
Data for board game Acquire with game ID: 5 acquired.
Data for board game Mare Mediterraneum with game ID: 6 acquired.
Data for board game Cathedral with game ID: 7 acquired.
Data for board game Lords of Creation with game ID: 8 acquired.
Data for board game El Caballero with game ID: 9 acquired.
Data for board game Elfenland with game ID: 10 acquired.
Data for board game Bohnanza with game ID: 11 acquired.
Data for board game Ra with game ID: 12 acquired.
Data for board game CATAN with game ID: 13 acquired.
Data for board game Basari with game ID: 14 acquired.
Data for board game Cosmic Encounter with game ID: 15 acquired.
Data for board game MarraCash with game ID: 16 acquired.
Data for board game Button Men with game ID: 17 acquired.
Data for board game Ro